In [1]:
from electric_meter import *
from datetime import datetime
from IPython.display import HTML

In [2]:
meter = read_data('data/meter.csv')

In [3]:
#meter = read_data('data/meter.csv')

costs, k, d = compute_costs(meter)

In [4]:
#costs.sort_index(ascending=True, inplace=True)

#from datetime import datetime
#costs.index = pd.to_datetime(costs.index).to_series().dt.strftime('%B %Y')

In [5]:
costs

,no_discount,pazgas_daytime,pazgas_unlimited,pazgas_weekend,pazgas_nighttime,amisragas_unlimited,electra_power,electra_hitec
timeperiod,,,,,,,,
פברואר 2023,361.867363,342.149977,343.773995,355.161377,351.456975,338.345984,343.773995,342.838494
מרץ 2023,893.244672,837.334342,848.582438,869.026290,875.452102,835.183768,848.582438,846.394499
אפריל 2023,1542.413667,1418.802158,1465.292983,1503.203855,1516.241981,1442.156778,1465.292983,1449.564256
מאי 2023,907.657708,852.874994,862.274822,885.840499,888.610310,848.659957,862.274822,861.367179
יוני 2023,1416.049182,1327.340830,1345.246723,1366.584418,1372.917636,1324.005985,1345.246723,1332.972647
יולי 2023,1901.456399,1787.293462,1806.383579,1845.581001,1854.090353,1777.861733,1806.383579,1802.029223
אוגוסט 2023,1844.831204,1731.446921,1752.589644,1797.289125,1796.429241,1724.917175,1752.589644,1746.113631


In [6]:
result = style_table(costs)
display(result)

,חחי,פזגז יום,פזגז ללא הגבלה,פזגז סופ״ש,פזגז לילה,אמישראגז,אלקטרה פאואר,אלקטרה הייטק
תקופה,,,,,,,,
פברואר 2023,361.87,342.15,343.77,355.16,351.46,338.35,343.77,342.84
מרץ 2023,893.24,837.33,848.58,869.03,875.45,835.18,848.58,846.39
אפריל 2023,1542.41,1418.80,1465.29,1503.20,1516.24,1442.16,1465.29,1449.56
מאי 2023,907.66,852.87,862.27,885.84,888.61,848.66,862.27,861.37
יוני 2023,1416.05,1327.34,1345.25,1366.58,1372.92,1324.01,1345.25,1332.97
יולי 2023,1901.46,1787.29,1806.38,1845.58,1854.09,1777.86,1806.38,1802.03
אוגוסט 2023,1844.83,1731.45,1752.59,1797.29,1796.43,1724.92,1752.59,1746.11


In [8]:
if False:
    cols = [costs.columns[i] for i in [0,1,5, 6]]
    display(style_table(costs[cols].copy()))

In [25]:
meter['month'] = meter.index.month


In [ ]:
v = meter.query("timeperiod == '20230601'")

#meter.groupby('month').mean(numeric_only=True).plot()
meter.groupby('hour').mean(numeric_only=True).plot()

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Create a pivot table for heatmap
heatmap_data = meter.pivot_table(index="month", columns="hour", values="consumption", aggfunc="sum")

# Create the heatmap using seaborn
plt.figure(figsize=(10, 6))
sns.heatmap(heatmap_data, annot=False, cmap="YlGnBu", fmt="")
plt.title("Consumption Heatmap")
plt.xlabel("Hour")
plt.ylabel("Month")
plt.show()


In [41]:
meter.tail()

,consumption,date,wday,hour,wday_name,timeperiod,month
time,,,,,,,
2023-08-31 19:00:00,9.444,2023-08-31,3,19,Thursday,20230801,8
2023-08-31 20:00:00,6.439,2023-08-31,3,20,Thursday,20230801,8
2023-08-31 21:00:00,2.266,2023-08-31,3,21,Thursday,20230801,8
2023-08-31 22:00:00,2.511,2023-08-31,3,22,Thursday,20230801,8
2023-08-31 23:00:00,2.347,2023-08-31,3,23,Thursday,20230801,8
